# Add ingredients to the fridge. 
This notebook can be called via the scheduling REST API to add a product to the fridge. 

It accepts two parameters: product and barcode. if a barcode is present and found in the database, then that product name will be used. otherwise the product name passed in will be used

> [!IMPORTANT]
> [Add a lakehouse](https://aka.ms/fabric/addlakehouse) to the notebook before first running it. **Failure to do so results in an error.**

Also, when calling via the API (i.e. not running it in the browser), then %pip magic won't work. So you will need to create a new Environment and use that in this notebook, that has the correct openai dll version installed to use the openai sdk.

In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
#%pip install openai==1.12.0
import openai
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StructType, StructField, TimestampType, StringType
from datetime import datetime

StatementMeta(, 864035bd-c0ae-46a0-8ced-e8405fa5433e, 5, Finished, Available)

### Parameter block
These can be overriden via the REST API or a Pipeline

In [2]:
product = "Unkonwn"
barcode = "4104420055421"

StatementMeta(, 864035bd-c0ae-46a0-8ced-e8405fa5433e, 6, Finished, Available)

### Lookup the barcode in our copy of the open food fact database
If not found, revert to the alternative product name passed in (the one generated via the AI Builder image detection plugin in Power Apps)

In [3]:
dfprod = spark.sql("Select product_name FROM OpenFoodFacts_LH.products_zip where code = " + barcode)
product_name = dfprod.first().product_name
if not product_name is None:
    product = product_name # found the product via barcode. use that instead of the default or passed in product description.

StatementMeta(, 864035bd-c0ae-46a0-8ced-e8405fa5433e, 7, Finished, Available)

### Initiate Open AI to estimate the expiry date
Using a low temperature of 0.2 to make sure the results are deterministic. For this kind of knowledge-search gpt 3.5 is more than sufficient to respond with believable expiry dates

In [4]:
ENDPOINT_URL = "https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1"
API_KEY =  mssparkutils.credentials.getSecret('https://kvaihackweu.vault.azure.net/','aihackapikey')  
API_VERSION = "2023-09-01-preview"
MODEL_NAME = "gpt-35-turbo"
import openai
from openai import AzureOpenAI



client = AzureOpenAI(
    azure_endpoint=ENDPOINT_URL,
    api_key=API_KEY,
    api_version=API_VERSION,
)

MESSAGES = [
    {"role": "system", "content": """you are a food concierge. your job is to tell me how long a product will last in my fridge or pantry. I will tell you the name of an edible product, 
you will simply respond to the amount of days that I can expect the product to be good to eat for from the date of purchase. 
for example if I ask "eggs" you would respond with "fridge: 20-30 days". 
Always respond in amount of days."""},
    {"role": "user", "content": "potatoes"},
    {
        "role": "assistant",
        "content": "pantry: 30 days",
    }
]


MESSAGES.append({"role": "user", "content": product})

completion = client.chat.completions.create(
    model=MODEL_NAME,  # e.g. gpt-35-instant
    messages=MESSAGES,
    temperature=0.2
)

#print(completion.model_dump_json(indent=2))
#print()
expirytext = completion.choices[0].message.content
print(expirytext)

StatementMeta(, 864035bd-c0ae-46a0-8ced-e8405fa5433e, 8, Finished, Available)

pantry: 6 months


## Cleanup the response
GPT will sometimes respond in weeks and months. so need to parse that and figure out the total days and create an expiry date.

In [ ]:
import re
from datetime import timedelta, date, datetime

if "-" in expirytext :
    m = re.findall(r"[0-9]+",expirytext)
    print (m)
    start = int(m[0])
    end = int(m[1])
    avg = (start + end) / 2
else:
    avg = int(re.findall(r"[0-9]+", expirytext)[0])

if "week" in expirytext:
    avg = avg * 7
if "month" in expirytext:
    avg = avg * 30

days = int(avg)
print(days)
expirydate = date.today()+timedelta(days=days)
print(expirydate)

### Save to DB
now we can create a new record with the estimated expiry date and append it to the fridgecontents table in the lakehouse

In [5]:
#spark = SparkSession.builder.getOrCreate()

dt = datetime.combine(expirydate, datetime.min.time())
data = [{"barcode": barcode, "name": product, "expirydate": dt}]

df = spark.createDataFrame(data)
df.show()
df.write.format("delta").mode("append").saveAsTable("fridgecontents")

StatementMeta(, b695ccdd-40d0-4200-a57c-55f8ca61949f, 13, Finished, Available)

+-------------+-------------------+-------+
|      barcode|         expirydate|   name|
+-------------+-------------------+-------+
|4104420055421|2024-02-16 00:00:00|Mandeln|
+-------------+-------------------+-------+

